# Llamaindex Model
<hr>

Download data

In [ ]:
! mkdir -p 'data/10k/'
! wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
! wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

<hr>

### Ingestion

**Environment setting**

In [37]:
import os

from dotenv import load_dotenv
load_dotenv()

import nest_asyncio

# Load environment variables from .env file
load_dotenv()

# Apply nest_asyncio if needed
nest_asyncio.apply()

# Set OpenAI API Key
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

# Set AWS Credentials
# os.environ['AWS_ACCESS_KEY_ID'] = os.getenv('AWS_ACCESS_KEY_ID')
# os.environ['AWS_SECRET_ACCESS_KEY'] = os.getenv('AWS_SECRET_ACCESS_KEY')

**Get the documents**

In [5]:
from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader('data/10k/').load_data()

In [6]:
print(len(documents))

545


**Ingestion pipeline**
- transformations
    - OpenAIEmbedding
        ```python
        embed_model = OpenAIEmbedding(
            model="text-embedding-3-large",
            dimensions=512,
        )
        ```

    - SentenceSplitter
        - chunk_size
        - chunk_overlap
        - tokenizer
        - paragraph_separator

    - TitleExtractor
        - LLM


In [7]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import TitleExtractor
from llama_index.core.ingestion import IngestionPipeline, IngestionCache

# create the pipeline with transformations
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=1024, chunk_overlap=20),
        OpenAIEmbedding(model='text-embedding-3-small'),
    ]
)

# run the pipeline
nodes = pipeline.run(documents=documents)

In [ ]:
nodes[0]

### Indexing

**Indexing and Storing in chromaDB**

- embed_model
    - OpenAI
        - text-embedding-ada-002 (Latest)
        - text-search-babbage-doc-001
        - text-similarity-davinci-001
        - text-embedding-3-small
    - HuggingFace
        - sentence-transformers/all-MiniLM-L6-v2
	    - sentence-transformers/all-mpnet-base-v2
	    - bert-base-uncased
        - BAAI/bge-small-en-v1.5

**First Time to create a collection in chroma_db**

In [12]:
import chromadb
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore

db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("10k_collection")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
embed_model = OpenAIEmbedding(model='text-embedding-3-small')

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

**Second Time extract from the Database**

In [10]:
import chromadb
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore

db2 = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db2.get_or_create_collection("10k_collection")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
embed_model = OpenAIEmbedding(model='text-embedding-3-small')
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embed_model,
)

### Query Interface

**Tools**

*Vector Tool*

In [11]:
from typing import List
from llama_index.core.vector_stores import FilterCondition
from llama_index.core.tools import FunctionTool
from llama_index.core.vector_stores import MetadataFilters

def vector_query(
    query : str,
    page_numbers: List[str],
) -> str:
    """Performs a vector search over an index.
    
    query (str): the string query to embeded.
    page_numbers (List[str]): Filter by set of pages. Leave BLANK if we want to search
    over all pages. Otherwise, filter by the set of sepcified pages.
    """

    metadata_dicts = [
        {"key": "page_label", "value": p}
        for p in page_numbers
    ]

    query_engine = index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)
    return response

vector_query_tool = FunctionTool.from_defaults(
    name="vector_query",
    fn=vector_query,
)

*summary tool*

In [12]:
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool

summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode='tree_summarize',
    use_async=True,
)
summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=(
        'Useful if you want to get a summary of the ragchecker.'
    )
)

In [13]:
docs = [
    "data/10k/uber_2021.pdf",
    "data/10k/lyft_2021.pdf",
]

search by tools

In [14]:
from pathlib import Path

documents_to_tools_dict = {}
for documents in docs:
    print(f"Getting tools for document: {documents}")
    documents_to_tools_dict[documents] = [vector_query_tool, summary_tool]

Getting tools for document: data/10k/uber_2021.pdf
Getting tools for document: data/10k/lyft_2021.pdf


put tools into a flat list

In [15]:
initial_tools = [t for documents in docs for t in documents_to_tools_dict[documents]]

In [25]:
len(initial_tools)  # should be 4

4

**Using agent worker**

define llm where can be changed

In [16]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model='gpt-3.5-turbo',temperature=0)

In [17]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    initial_tools,
    llm=llm,
    verbose=True,
)

agent = AgentRunner(agent_worker)

<hr>

### Test case (gpt-4o)

In [33]:
response = agent.query(
    "Tell me the diffrerence between Uber and Lyft in"
    "The impact of covid-19.",
)

Added user message to memory: Tell me the diffrerence between Uber and Lyft inThe impact of covid-19.
=== Calling Function ===
Calling function: vector_query with args: {"query": "Difference between Uber and Lyft in the impact of covid-19", "page_numbers": []}
=== Function Output ===
Uber and Lyft both faced challenges due to the COVID-19 pandemic, impacting their operations and financial performance. However, Lyft specifically mentioned challenges related to delays in manufacturing assets, increased costs, and supply chain issues affecting the deployment of new vehicles and features on their network. Lyft also highlighted the adverse impact on demand for vehicles rented through their Express Drive program and Lyft Rentals, leading to operational changes and cost increases. On the other hand, Uber emphasized the global impact of COVID-19 on drivers, merchants, consumers, and business partners, leading to reduced demand for mobility rides and driver supply constraints. Uber responded by

In [38]:
response = agent.query(
    "Give me a summary of the document lyft."
    "and what is the 10k analysis overall based on uber.",
)

print(str(response))

Added user message to memory: Give me a summary of the document lyft.and what is the 10k analysis overall based on uber.
=== Calling Function ===
Calling function: summary_tool with args: {"input": "lyft"}
=== Function Output ===
Lyft, Inc. is a transportation network company that operates a ridesharing marketplace connecting drivers with riders through its mobile application. The company offers various transportation options, including ridesharing, bike and scooter sharing, public transit integration, and car rentals. Lyft is committed to improving people's lives by providing affordable and reliable transportation options. The company has faced challenges due to the COVID-19 pandemic, impacting its operations, financial performance, and driver availability. Additionally, Lyft has focused on safety measures, diversity in its workforce, and environmental initiatives, such as transitioning to electric vehicles.
=== Calling Function ===
Calling function: summary_tool with args: {"input": 

<hr>

### Evaluation

**RagChecker**

In [ ]:
! pip install ipywidgets


In [2]:
from ragchecker.integrations.llama_index import response_to_rag_results
from ragchecker import RAGResults, RAGChecker
from ragchecker.metrics import all_metrics

**Using RAGChecker with LlamaIndex**

In [28]:
# rag_application = index.as_query_engine()

In [33]:
# User query and groud truth answer
user_query = "How many employees did Uber have globally as of December 31, 2021?"
gt_answer = "Uber had approximately 29,300 employees globally as of December 31, 2021"

# Get response from LlamaIndex
response_object = agent.query(user_query)
#response_object = rag_application.query(user_query)

# Convert to RAGChecker format
rag_result = response_to_rag_results(
    query=user_query,
    gt_answer=gt_answer,
    response_object=response_object,
)

# Create RAGResults object
rag_results = RAGResults.from_dict({"results": [rag_result]})
print(rag_results)

Added user message to memory: How many employees did Uber have globally as of December 31, 2021?
=== Calling Function ===
Calling function: vector_query with args: {"query": "number of employees at Uber as of December 31, 2021", "page_numbers": []}
=== Function Output ===
Approximately 29,300 employees globally.
=== LLM Response ===
Uber had approximately 29,300 employees globally as of December 31, 2021.
RAGResults(
  1 RAG results,
  Metrics:
  {
    "overall_metrics": {},
    "retriever_metrics": {},
    "generator_metrics": {}
  }
)


/Users/chris/Desktop/7980/CS7980_Capstone_RBCMuseum/model/venv/lib/python3.11/site-packages/dataclasses_json/core.py:201: RuntimeWarning: 'NoneType' object value of non-optional type query_id detected when decoding RAGResult.
  warnings.warn(


In [34]:
print(response_object)

Uber had approximately 29,300 employees globally as of December 31, 2021.


In [35]:
# Initialize RAGChecker
evaluator = RAGChecker(
    extractor_name="bedrock/meta.llama3-70b-instruct-v1:0",
    checker_name="bedrock/meta.llama3-70b-instruct-v1:0",
    batch_size_extractor=32,
    batch_size_checker=32,
)

# Evaluate using RAGChecker
evaluator.evaluate(rag_results, all_metrics)

# Print detailed results
print(rag_results)

2024-10-03 10:12:06.020 | INFO     | ragchecker.evaluator:extract_claims:113 - Extracting claims for gt_answer of 1 RAG results.
  0%|          | 0/1 [00:02<?, ?it/s]

Exit with the following error:
'NoCredentialsError' object has no attribute 'response'


TypeError: 'NoneType' object is not iterable

<hr>

**Ragas**

In [ ]:
! pip install ragas

Data preparing

In [87]:
data = {
    "question": [
        "What was Lyft's total revenue for the year 2021?",
        "What strategic partnerships did Uber leverage for autonomous vehicles in 2021?",
        "How many patents did Lyft hold as of December 31, 2021?",
        "What were Uber’s primary operating segments in 2021?",
        "How many employees did Uber have globally as of December 31, 2021?",
        "What commitment has Lyft made regarding electric vehicles by 2030?",
        "What acquisition did Uber complete in 2021 related to alcohol delivery?",
        "What was Lyft's market value of common stock held by non-affiliates as of June 30, 2021?",
        "What impact did COVID-19 have on Lyft's business in 2021?",
        "Which key technology partners did Lyft collaborate with in its cloud infrastructure?",
        "Summarize the impact of COVID-19 on Uber’s operations during 2021.",
        "How did Lyft address its commitment to climate change and sustainability in 2021?",
        "What were Uber’s primary areas of growth strategy in 2021?",
        "Summarize Lyft's business model and revenue generation approach in 2021.",
        "How did Uber enhance its platform's operational efficiency in 2021?",
    ],
    "ground_truth": [
        "Lyft's total revenue for the year 2021 was $3.21 billion.",
        "Uber partnered with Motional and Argo AI in 2021 to deploy autonomous vehicles on its platform.",
        "Lyft held 343 issued U.S. patents and had 310 U.S. patent applications pending as of December 31, 2021.",
        "Uber’s primary operating segments in 2021 were Mobility, Delivery, and Freight.",
        "Uber had approximately 29,300 employees globally as of December 31, 2021.",
        "Lyft committed to having 100% electric vehicles on its platform by 2030.",
        "Uber acquired The Drizly Group, Inc. in 2021 to expand its alcohol delivery offerings.",
        "Lyft's market value of common stock held by non-affiliates as of June 30, 2021, was $19.6 billion.",
        "COVID-19 led to decreased demand on Lyft's platform, impacting its revenues and drivers’ earning opportunities.",
        "Lyft collaborated with Amazon Web Services (AWS) for its cloud services.",
        "COVID-19 affected Uber’s operations in 2021, leading to reduced Mobility demand due to travel restrictions, though its Delivery segment saw growth.",
        "Lyft committed to reaching 100% electric vehicles on its platform by 2030 as part of its broader efforts toward sustainability and addressing climate change.",
        "Uber’s growth strategy in 2021 included expanding its platform offerings, investing in new technology, growing its Delivery and Freight businesses, and leveraging synergies across its segments.",
        "Lyft’s business model in 2021 focused on connecting riders with drivers through its ridesharing platform, generating revenue from service fees and commissions. It also expanded its offerings through Light Vehicles, public transit integration, and Lyft Rentals.",
        "Uber enhanced operational efficiency in 2021 by leveraging its technology to improve demand prediction, matching, and dispatching, which reduced costs and optimized trip utilization across its platform.",
    ]
}


Step 1: create a pandas DataFrame

In [88]:
import pandas as pd

# Create a pandas DataFrame
df = pd.DataFrame(data)

convert data form

In [90]:
import pandas as pd
from datasets import Dataset  # Hugging Face Dataset library

testset = Dataset.from_pandas(df)

In [93]:
ds_dict = testset.to_dict()

In [95]:
hf_dataset = Dataset.from_dict(ds_dict)

Step 2 Save the work to local disc

In [44]:
output_path = "./test_db"  # Specify your desired output directory
os.makedirs(output_path, exist_ok=True)  # Create directory if it doesn't exist


In [45]:
# Save as CSV
testset.to_pandas().to_csv(os.path.join(output_path, "10K_testset.csv"), index=False)

# Save as JSON
testset.to_pandas().to_json(os.path.join(output_path, "10K_testset.json"), orient="records", lines=True)

Step 3: Test with the general query engine

In [ ]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)
from ragas.integrations.llama_index import evaluate


# Initialize evaluator LLM
evaluator_llm = OpenAI(model="gpt-3.5-turbo")

# Define the metrics for evaluation
metrics = [faithfulness, answer_relevancy, context_precision, context_recall]

result = evaluate(
    query_engine=index.as_query_engine(),
    metrics=metrics,
    dataset=hf_dataset,
    llm=evaluator_llm,
    embeddings=OpenAIEmbedding(),
)

In [167]:
print(result)

{'faithfulness': 0.8711, 'answer_relevancy': 0.9786, 'context_precision': 0.9667, 'context_recall': 0.8667}


In [168]:
result.to_pandas()

,question,contexts,answer,ground_truth,faithfulness,answer_relevancy,context_precision,context_recall
0,What was Lyft's total revenue for the year 2021?,[Significant items subject to estimates and as...,"Lyft's total revenue for the year 2021 was $3,...",Lyft's total revenue for the year 2021 was $3....,1.000000,1.000000,1.0,1.0
1,What strategic partnerships did Uber leverage ...,[Through our partnership with Arizona State Un...,Uber leveraged a strategic partnership with Ar...,Uber partnered with Motional and Argo AI in 20...,0.000000,0.965757,0.5,0.0
2,How many patents did Lyft hold as of December ...,[We have invested in a patent program to ident...,Lyft held 343 issued U.S. patents and had 70 i...,Lyft held 343 issued U.S. patents and had 310 ...,1.000000,0.989130,1.0,1.0
3,What were Uber’s primary operating segments in...,[which are miles driven by carriers between sh...,Uber's primary operating segments in 2021 were...,Uber’s primary operating segments in 2021 were...,0.500000,0.998946,1.0,1.0
4,How many employees did Uber have globally as o...,"[the EU, the United States, Brazil, and ...","Approximately 29,300 employees globally as of ...","Uber had approximately 29,300 employees global...",1.000000,1.000000,1.0,1.0
5,What commitment has Lyft made regarding electr...,[Because we stand at a pivotal moment in the f...,Lyft has made the commitment to reach 100% ele...,Lyft committed to having 100% electric vehicle...,1.000000,0.988208,1.0,1.0
6,What acquisition did Uber complete in 2021 rel...,[Financial and Operational HighlightsYear Ende...,Uber completed the acquisition of 100% ownersh...,"Uber acquired The Drizly Group, Inc. in 2021 t...",1.000000,0.920738,1.0,1.0
7,What was Lyft's market value of common stock h...,[UNITED STATESSECURITIES AND EXCHANGE COMMISSI...,Lyft's market value of common stock held by no...,Lyft's market value of common stock held by no...,1.000000,1.000000,1.0,1.0
8,What impact did COVID-19 have on Lyft's busine...,[We also design and contract to manufacture ce...,COVID-19 had a significant impact on Lyft's bu...,COVID-19 led to decreased demand on Lyft's pla...,1.000000,1.000000,1.0,1.0
9,Which key technology partners did Lyft collabo...,[•Selection and Convenience. We designed the ...,Lyft collaborated with Amazon Web Services (AW...,Lyft collaborated with Amazon Web Services (AW...,1.000000,0.965228,1.0,1.0


Step 4: Query each question using query_engine I customized

In [48]:
responses = []
for question in ds_dict["question"]:
    response = agent.query(question)  # Querying each question
    responses.append(response)

# Add the responses to the dataset
df["response"] = responses

Added user message to memory: What was Lyft's total revenue for the year 2021?
=== Calling Function ===
Calling function: vector_query with args: {"query": "Lyft total revenue 2021", "page_numbers": ["1"]}
=== Function Output ===
Lyft's total revenue for 2021 was not explicitly provided in the context information.
=== LLM Response ===
It seems that the total revenue for Lyft in the year 2021 was not explicitly mentioned in the available information. If you would like, I can attempt to provide a summary of the information related to Lyft. Would you like me to do that?
Added user message to memory: What strategic partnerships did Uber leverage for autonomous vehicles in 2021?
=== Calling Function ===
Calling function: vector_query with args: {"query": "strategic partnerships for autonomous vehicles in 2021", "page_numbers": ["16"]}
=== Function Output ===
Uber has not mentioned any specific strategic partnerships for autonomous vehicles in 2021 in the provided context information.
=== LL

In [56]:
responses[0]

Response(response='It seems that the total revenue for Lyft in the year 2021 was not explicitly mentioned in the available information. If you would like, I can attempt to provide a summary of the information related to Lyft. Would you like me to do that?', source_nodes=[NodeWithScore(node=TextNode(id_='3e363fdf-b136-4b51-a952-c6b2649eb9b6', embedding=None, metadata={'page_label': '1', 'file_name': 'lyft_2021.pdf', 'file_path': '/Users/chris/Desktop/7980/CS7980_Capstone_RBCMuseum/model/data/10k/lyft_2021.pdf', 'file_type': 'application/pdf', 'file_size': 1440303, 'creation_date': '2024-10-03', 'last_modified_date': '2024-10-03'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='011ba502-fcc1-41b7-9131-180f5e

In [57]:
responses[0].response

'It seems that the total revenue for Lyft in the year 2021 was not explicitly mentioned in the available information. If you would like, I can attempt to provide a summary of the information related to Lyft. Would you like me to do that?'

In [59]:
extracted_responses = [response.response for response in responses]

In [60]:
extracted_responses[0]

'It seems that the total revenue for Lyft in the year 2021 was not explicitly mentioned in the available information. If you would like, I can attempt to provide a summary of the information related to Lyft. Would you like me to do that?'

In [146]:
# Iterate over each response in the responses
extracted_metadata = []

for response in responses:
    metadata_list = []
    
    # Iterate over each source node in the response
    for node_with_score in response.source_nodes:
        text_node = node_with_score.node  # Get the TextNode
        metadata_list.append(text_node.metadata)  # Extract the metadata
    
    # Append the list of metadata for this response
    extracted_metadata.append(metadata_list)


In [147]:
len(extracted_metadata)

15

In [149]:
extracted_metadata[1]

[{'page_label': '16',
  'file_name': 'uber_2021.pdf',
  'file_path': '/Users/chris/Desktop/7980/CS7980_Capstone_RBCMuseum/model/data/10k/uber_2021.pdf',
  'file_type': 'application/pdf',
  'file_size': 1880483,
  'creation_date': '2024-10-03',
  'last_modified_date': '2024-10-03'},
 {'page_label': '16',
  'file_name': 'uber_2021.pdf',
  'file_path': '/Users/chris/Desktop/7980/CS7980_Capstone_RBCMuseum/model/data/10k/uber_2021.pdf',
  'file_type': 'application/pdf',
  'file_size': 1880483,
  'creation_date': '2024-10-03',
  'last_modified_date': '2024-10-03'}]

In [156]:
extracted_metadata_as_strings = [
    [str(metadata) for metadata in metadata_list]  # Convert each metadata dict to a string
    for metadata_list in extracted_metadata  # Iterate over each response's metadata list
]

In [157]:
len(extracted_metadata_as_strings)

15

In [158]:
extracted_metadata_as_strings[0]

["{'page_label': '1', 'file_name': 'lyft_2021.pdf', 'file_path': '/Users/chris/Desktop/7980/CS7980_Capstone_RBCMuseum/model/data/10k/lyft_2021.pdf', 'file_type': 'application/pdf', 'file_size': 1440303, 'creation_date': '2024-10-03', 'last_modified_date': '2024-10-03'}",
 "{'page_label': '1', 'file_name': 'uber_2021.pdf', 'file_path': '/Users/chris/Desktop/7980/CS7980_Capstone_RBCMuseum/model/data/10k/uber_2021.pdf', 'file_type': 'application/pdf', 'file_size': 1880483, 'creation_date': '2024-10-03', 'last_modified_date': '2024-10-03'}"]

In [169]:
# List to hold the contexts for each response
contexts = []

# Iterate over each response in the responses
for response in responses:
    # Extract the text from each source_node (context that was fed to the LLM)
    context_for_response = [node.node.text for node in response.source_nodes]
    
    # Append the context (a list of text) for this response
    contexts.append(context_for_response)

In [170]:
len(contexts)

15

In [171]:
contexts[0]

['UNITED STATESSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\nFORM 10-K\n \n(Mark One)\n☒\nANNUAL REPORT PURS UANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended December 31, 2021\nOR\n☐\nTRANSITION REPORT PURS UANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 FOR THE TRANSITION PERIODFROM                      TO\nCommission File Number 001-38846\nLyft, Inc.\n(Exact name of Registrant as specified in i\nts Charter)Delaware\n20-8809830 (State or other jurisdiction of\nincorporation or organization)\n(I.R.S. EmployerIdentification No.)\n185 Berry Street, Suite 5000\nSan Francisco, California\n94107 (Address of principal executive offices)\n(Zip Code) Registrant’s telephone number, including area code: (844) 250\n-2773Securities registered pursuant to Section 12(b) of the Act: \nTitle of each class\nTradingSymbol(s)\nName of each exchange on which registered Class A common stock, par value of $0.00001 per share\nLYFT Na

In [172]:
# Extract and convert the context (text from source_nodes) to strings
extracted_context_as_strings = [
    [str(node.node.text) for node in response.source_nodes]  # Convert each source_node's text to a string
    for response in responses  # Iterate over each response
]

In [173]:
len(extracted_context_as_strings)

15

In [174]:
import pandas as pd

# Create a pandas DataFrame
data = {
        "question": [
        "What was Lyft's total revenue for the year 2021?",
        "What strategic partnerships did Uber leverage for autonomous vehicles in 2021?",
        "How many patents did Lyft hold as of December 31, 2021?",
        "What were Uber’s primary operating segments in 2021?",
        "How many employees did Uber have globally as of December 31, 2021?",
        "What commitment has Lyft made regarding electric vehicles by 2030?",
        "What acquisition did Uber complete in 2021 related to alcohol delivery?",
        "What was Lyft's market value of common stock held by non-affiliates as of June 30, 2021?",
        "What impact did COVID-19 have on Lyft's business in 2021?",
        "Which key technology partners did Lyft collaborate with in its cloud infrastructure?",
        "Summarize the impact of COVID-19 on Uber’s operations during 2021.",
        "How did Lyft address its commitment to climate change and sustainability in 2021?",
        "What were Uber’s primary areas of growth strategy in 2021?",
        "Summarize Lyft's business model and revenue generation approach in 2021.",
        "How did Uber enhance its platform's operational efficiency in 2021?",
    ],
    "ground_truth": [
        "Lyft's total revenue for the year 2021 was $3.21 billion.",
        "Uber partnered with Motional and Argo AI in 2021 to deploy autonomous vehicles on its platform.",
        "Lyft held 343 issued U.S. patents and had 310 U.S. patent applications pending as of December 31, 2021.",
        "Uber’s primary operating segments in 2021 were Mobility, Delivery, and Freight.",
        "Uber had approximately 29,300 employees globally as of December 31, 2021.",
        "Lyft committed to having 100% electric vehicles on its platform by 2030.",
        "Uber acquired The Drizly Group, Inc. in 2021 to expand its alcohol delivery offerings.",
        "Lyft's market value of common stock held by non-affiliates as of June 30, 2021, was $19.6 billion.",
        "COVID-19 led to decreased demand on Lyft's platform, impacting its revenues and drivers’ earning opportunities.",
        "Lyft collaborated with Amazon Web Services (AWS) for its cloud services.",
        "COVID-19 affected Uber’s operations in 2021, leading to reduced Mobility demand due to travel restrictions, though its Delivery segment saw growth.",
        "Lyft committed to reaching 100% electric vehicles on its platform by 2030 as part of its broader efforts toward sustainability and addressing climate change.",
        "Uber’s growth strategy in 2021 included expanding its platform offerings, investing in new technology, growing its Delivery and Freight businesses, and leveraging synergies across its segments.",
        "Lyft’s business model in 2021 focused on connecting riders with drivers through its ridesharing platform, generating revenue from service fees and commissions. It also expanded its offerings through Light Vehicles, public transit integration, and Lyft Rentals.",
        "Uber enhanced operational efficiency in 2021 by leveraging its technology to improve demand prediction, matching, and dispatching, which reduced costs and optimized trip utilization across its platform.",
    ],
    "contexts": extracted_context_as_strings,
    "answer": extracted_responses
} 
df = pd.DataFrame(data)

In [175]:
testset = Dataset.from_pandas(df)
ds_dict = testset.to_dict()
hf_dataset_with_respones = Dataset.from_dict(ds_dict)

In [176]:
hf_dataset_with_respones[0]

{'question': "What was Lyft's total revenue for the year 2021?",
 'ground_truth': "Lyft's total revenue for the year 2021 was $3.21 billion.",
 'contexts': ['UNITED STATESSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\nFORM 10-K\n \n(Mark One)\n☒\nANNUAL REPORT PURS UANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended December 31, 2021\nOR\n☐\nTRANSITION REPORT PURS UANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 FOR THE TRANSITION PERIODFROM                      TO\nCommission File Number 001-38846\nLyft, Inc.\n(Exact name of Registrant as specified in i\nts Charter)Delaware\n20-8809830 (State or other jurisdiction of\nincorporation or organization)\n(I.R.S. EmployerIdentification No.)\n185 Berry Street, Suite 5000\nSan Francisco, California\n94107 (Address of principal executive offices)\n(Zip Code) Registrant’s telephone number, including area code: (844) 250\n-2773Securities registered pursuant to Section 12(b) o

Step 4: Convert the modified DataFrame back to HF Dataset and dict

In [177]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)
from ragas import evaluate


# Initialize evaluator LLM
evaluator_llm = OpenAI(model="gpt-3.5-turbo")

# Define the metrics for evaluation
metrics = [faithfulness, answer_relevancy, context_precision, context_recall]

result = evaluate(
    metrics=metrics,
    dataset=hf_dataset_with_respones,
)

Evaluating:  27%|██▋       | 16/60 [00:06<00:21,  2.03it/s]Exception raised in Job[52]: BadRequestError(Error code: 400 - {'error': {'message': "Invalid 'messages[0].content': string too long. Expected a string with maximum length 1048576, but got a string with length 2268666 instead.", 'type': 'invalid_request_error', 'param': 'messages[0].content', 'code': 'string_above_max_length'}})
Exception raised in Job[44]: BadRequestError(Error code: 400 - {'error': {'message': "Invalid 'messages[0].content': string too long. Expected a string with maximum length 1048576, but got a string with length 2275291 instead.", 'type': 'invalid_request_error', 'param': 'messages[0].content', 'code': 'string_above_max_length'}})
Evaluating: 100%|██████████| 60/60 [03:07<00:00,  3.13s/it]


In [178]:
print(result)

{'faithfulness': 0.4615, 'answer_relevancy': 0.3020, 'context_precision': 0.2692, 'context_recall': 0.3077}


In [179]:
result.to_pandas()

,question,ground_truth,contexts,answer,faithfulness,answer_relevancy,context_precision,context_recall
0,What was Lyft's total revenue for the year 2021?,Lyft's total revenue for the year 2021 was $3....,[UNITED STATESSECURITIES AND EXCHANGE COMMISSI...,It seems that the total revenue for Lyft in th...,0.5,0.000000,0.0,0.0
1,What strategic partnerships did Uber leverage ...,Uber partnered with Motional and Argo AI in 20...,[expressed an intention to file) arbitration d...,Uber has not mentioned any specific strategic ...,0.5,0.000000,0.0,0.0
2,How many patents did Lyft hold as of December ...,Lyft held 343 issued U.S. patents and had 310 ...,[entrants in the market that may be well-estab...,The number of patents held by Lyft as of Decem...,0.5,0.000000,0.0,0.0
3,What were Uber’s primary operating segments in...,Uber’s primary operating segments in 2021 were...,[UNITED STATESSECURITIES AND EXCHANGE COMMISSI...,Uber's primary operating segments in 2021 were...,0.0,0.000000,0.0,0.0
4,How many employees did Uber have globally as o...,"Uber had approximately 29,300 employees global...",[Through our partnership with Arizona State Un...,"Uber had approximately 29,300 employees global...",1.0,1.000000,0.5,1.0
5,What commitment has Lyft made regarding electr...,Lyft committed to having 100% electric vehicle...,[PART IITEM 1. BUSINESS\nOverview\nUber\n Tech...,I couldn't find specific information about Lyf...,0.0,0.000000,0.0,0.0
6,What acquisition did Uber complete in 2021 rel...,"Uber acquired The Drizly Group, Inc. in 2021 t...",[expressed an intention to file) arbitration d...,It seems that there is no specific information...,0.5,0.000000,0.0,0.0
7,What was Lyft's market value of common stock h...,Lyft's market value of common stock held by no...,[],I couldn't find the specific information on Ly...,0.0,0.000000,0.0,0.0
8,What impact did COVID-19 have on Lyft's busine...,COVID-19 led to decreased demand on Lyft's pla...,[Impact of COVID-19 to our BusinessThe\n ongoi...,The impact of COVID-19 on Lyft's business in 2...,1.0,0.000000,1.0,1.0
9,Which key technology partners did Lyft collabo...,Lyft collaborated with Amazon Web Services (AW...,[PART IITEM 1. BUSINESS\nOverview\nUber\n Tech...,I couldn't find specific information about Lyf...,0.0,0.000000,0.0,0.0


**Have not done yet...**